# Исследование надежности заемщиков.

Кредитный отдел банка заказал провеение анализа статистики платежеспособности клиентов,а именно влияние таких факторов как семейное положение и количество детей клиента на факт погашения кредита в срок.

Данное исследование ляжет в основу при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

In [ ]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
display(data.head(20))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Вывел на экран 20 первых строк датафрейма, чтобы понять с каким данными необходимо будет работать.

### Шаг 2.1 Заполнение пропусков

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Информация о датафрейме показывает, что вероятнее всего пропуски в столбцах days_employed & total_income.

In [ ]:
omissions = data['days_employed'].isna().sum() / 21525 *100
print('Доля пропусков в двух столбцах', omissions,'%')

Доля пропусков в двух столбцах 10.099883855981417 %


В двух столбцах пропуски составляют немного более 10%, что очевидно много и я не могу игнорировать  или просто удалить строки, так как это исказит результат.

Так как пропущенные данные связаны со временем работы и зароботной платой, вероятнее всего, некоторые заемщики просто не предоставили подтверждающий документ с места работы. Наилучшим способом заполнить пропуски будет использование медианных значений, так как оно наиболее реально отражает срединный показатель и не подвержен значительным всплескам в показателях.

In [ ]:
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
#data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Медианная зарблата составила около 145 тысяч, что похоже не реальность, однако медианное значение days_employed составило -1203, что явно не отображает действительность. Очевидно в указанном стобце есть аномалия связанная отрицательными значениями. После исправления этой аномалии логичным шагом будет пересчитать медианнные показатели и заполнить пропуски снова.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [ ]:
data.loc[data['days_employed'] < 0, 'days_employed'] = data['days_employed'] * (-1)
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
display(data.tail(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Убрал отрицательные числа в колличестве дней и назначил новую медиану для заполнения пропусков. Тем не менее у многих людей старше 50 лет однозначно завышенной число рабочих дней!!!

### Шаг 2.3. Изменение типов данных.

In [ ]:
data['total_income'] = data['total_income'].astype(int)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Заменил вещественный тип данных в столбце total_income на цифровой.

### Шаг 2.4. Удаление дубликатов.

In [ ]:
print('В датафрейме', data.duplicated().sum(), 'дубликатов')

В датафрейме 54 дубликатов


Удалим дубликаты не создавая столбец с старыми индексами.

In [ ]:

data = data.drop_duplicates().reset_index(drop=True)

Для поиска неявных дубликатов выберем все уникальный значения в столбце education.

In [ ]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Очевидно, появление неявных дубликатов вязаны с использованием разных регистров букв при переносе данных из анкет заемщиков. Исправим используя метод str.lower() 

In [ ]:
data['education'] = data['education'].str.lower()

И проверим

In [ ]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Проверим уникальные данные в столбцах family_status, income_type, purpose

In [ ]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [ ]:
data['income_type'].unique()


array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Ну думаю что колличество детей может быть -1.
Заменим на 1.

In [ ]:
data.loc[data['children'] == -1, 'children'] = 1

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В столбце purpose так же встречаются похожие цели, который можно объединить в одну категорию. Появление такого количества схожих наименований, вероятно, связано с произвольной формой написания целей кредитования в анкете заемщика.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создаем новые датафреймы для классификации education и family_status и удаляем все явные дубликаты. 

In [ ]:
df_edu = pd.DataFrame(data[['education', 'education_id']])
df_fam = pd.DataFrame(data[['family_status', 'family_status_id']])
df_edu = df_edu.drop_duplicates().reset_index(drop=True)
display(df_edu)
df_fam = df_fam.drop_duplicates().reset_index(drop=True)
display(df_fam)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


В новых таблицах можно быстро получить информацию по идентефикатору.

Удалим столбцы education и family_status из базовых данных.

In [ ]:
data = data.drop(['education', 'family_status'], axis=1)
display(data)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем
21467,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21468,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость
21469,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


Отлично! Осталось 10 столбцов.

### Шаг 2.6. Категоризация дохода.

Создадим новый столбец в базавой таблице total_income_category для категоризации дохода.
Для этого создадим функцию income_group и примением ее к таблице через apply().

In [ ]:
def income_group(row):
    total_income = row['total_income']
    if total_income < 30000:
        return 'E'
    if 30001 < total_income < 50000:
        return 'D'
    if 50001 < total_income < 200000:
        return 'C'
    if 200001 < total_income < 1000000:
        return 'B'
    return 'A'

data['total_income_category'] = data.apply(income_group, axis=1)
display(data)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B
21467,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21468,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C
21469,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B


### Шаг 2.7. Категоризация целей кредита.

In [ ]:
def purpose_category(row):
    purpose = row['purpose']
    if ('автомобил' in purpose):
        return 'операции с автомобилем'
    if ('недвижим' in purpose or 'жил' in purpose):
        return 'операции с недвижимостью'
    if ('свадьб' in purpose):
        return 'проведение свадьбы'
    if ('образов' in purpose):
        return 'получение образования'
    
data['purpose_category'] = data.apply(purpose_category, axis=1)
display(data)


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21467,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21468,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21469,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


Проверим работу функции.

In [ ]:
data['purpose_category'].isna().sum()

0

### Ответы на вопросы.

Попробуем подсчитать связь количества детей с возвратом кредита в срокю
Возьмем процентное соотношение просроченных кредитов к общему числу по каждой категории.

Соберем небольшую таблицу из полученных данных.

In [ ]:
#общее количество кредитов выданных для каждой категории
data_pivot_children_common = data.pivot_table(index=['children'], values = ['debt'], aggfunc='count') 
#число случаев задолженности по кредитам для каждой категории подсчитаем методом sum() 
#так как случаи просрочки по займам отмечены True(1)
data_pivot_children = data.pivot_table(index=['children'], values = ['debt'], aggfunc='sum') 

display(data_pivot_children)
display(data_pivot_children_common)


,debt
children,
0,1063
1,445
2,194
3,27
4,4
5,0
20,8


,debt
children,
0,14107
1,4856
2,2052
3,330
4,41
5,9
20,76


In [ ]:
data_children_df = {'Children': ['0', '1', '2', '3', '4', '5', '20'], 
        'loan_count': [14107, 4586, 2052, 330, 41, 9, 76], 
       'debt': [1063, 445, 194, 27, 4, 0, 8]}
children_df = pd.DataFrame(data_children_df)
children_df['non_return'] = children_df['debt'] / children_df['loan_count']
display(children_df)


,Children,loan_count,debt,non_return
0,0,14107,1063,0.075353
1,1,4586,445,0.097034
2,2,2052,194,0.094542
3,3,330,27,0.081818
4,4,41,4,0.097561
5,5,9,0,0.000000
6,20,76,8,0.105263


##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?


Процентное соотношение просроченных кредитов по категориям связзанным с количеством детей примерно одинаковое и колеблется в пределах от 7,5% у клиентов без детей до 10% у клиентов в 20 детьми. Исключением являются заемщики с 5 детьми имея стопроцентный возврат в срок. Очевидно эта категория является самой надежной, однако принимать этот показатель во внимание следует осторожно, так как датафрейм содержит информацию только о девяти таких случаяч, что, вероятнее всего, слишком мало чтобы ссылаться на эти данные.



##### Вывод 1:

Зависимость между количеством детей и возвратом кредит в срок НЕ значительна.

Наиболее надежные клиенты, это клиенты неимеющие детей, либо клиенты имеющие либо 3-х либо 5-х детей.
Худший показатель по указанной категории - это заемщики с 20 детьми.

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

Попробуем собрать похожую таблицу.

In [ ]:
data_pivot_fam_common = data.pivot_table(index=['family_status_id'], values = ['debt'], aggfunc='count') 
data_pivot_fam = data.pivot_table(index=['family_status_id'], values = ['debt'], aggfunc='sum') 

display(data_pivot_fam)
display(data_pivot_fam_common)


,debt
family_status_id,
0,931
1,388
2,63
3,85
4,274


,debt
family_status_id,
0,12344
1,4163
2,959
3,1195
4,2810


In [ ]:
data_fam_df = {'family_id': ['0', '1', '2', '3', '4'], 
        'loan_count': [12344, 4163, 959, 1195, 2810], 
       'debt': [931, 388, 63, 85, 274]}
family_df = pd.DataFrame(data_fam_df)
family_df['non_return'] = family_df['debt'] / family_df['loan_count']
display(family_df)

,family_id,loan_count,debt,non_return
0,0,12344,931,0.075421
1,1,4163,388,0.093202
2,2,959,63,0.065693
3,3,1195,85,0.071130
4,4,2810,274,0.097509


##### Вывод 2:

Самые надежные клиенты в этой категорий имеют индекс 2 (вдовец/вдова) с всего-лишь 6.5% невозврата. Так же хороший показатель у категории 3 (в разводе) около 7.1%.

Худшие показатели у категории 4 (не женат/не замужем) около 9.7% и категории 1 (гражданский брак) около 9.3% невозвратов.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data_pivot_income_common = data.pivot_table(index=['total_income_category'], values = ['debt'], aggfunc='count') 
data_pivot_income = data.pivot_table(index=['total_income_category'], values = ['debt'], aggfunc='sum') 

display(data_pivot_income)
display(data_pivot_income_common)


,debt
total_income_category,
A,2
B,356
C,1360
D,21
E,2


,debt
total_income_category,
A,26
B,5040
C,16033
D,350
E,22


In [ ]:
data_income_df = {'income_id': ['A', 'B', 'C', 'D', 'E'], 
        'loan_count': [26, 5040, 16033, 350, 22], 
       'debt': [2, 356, 1360, 21, 2]}
income_df = pd.DataFrame(data_income_df)
income_df['non_return'] = income_df['debt'] / income_df['loan_count']
display(income_df)

,income_id,loan_count,debt,non_return
0,A,26,2,0.076923
1,B,5040,356,0.070635
2,C,16033,1360,0.084825
3,D,350,21,0.060000
4,E,22,2,0.090909


##### Вывод 3:

Неожиданно заемщики с достаточно низким доходом из категории D (30001 - 50000) являются самыми надежными с минимальними значениями невозвратов 6%, в то время как заемщики из категорий E (0 - 30000) % и C (50001 - 200000) имеют около 9% и 8.4% невозвратов соответственно.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data_pivot_purpose_common = data.pivot_table(index=['purpose_category'], values = ['debt'], aggfunc='count') #общее количество кредитов выданных
data_pivot_purpose = data.pivot_table(index=['purpose_category'], values = ['debt'], aggfunc='sum') #число случаев задолженности по кредитам

display(data_pivot_purpose)
display(data_pivot_purpose_common)


,debt
purpose_category,
операции с автомобилем,403
операции с недвижимостью,782
получение образования,370
проведение свадьбы,186


,debt
purpose_category,
операции с автомобилем,4308
операции с недвижимостью,10814
получение образования,4014
проведение свадьбы,2335


In [ ]:
data_purpose_df = {'family_id': ['операции с автомобилем', 'операции с недвижимостью', 
                                 'получение образования', 'проведение свадьбы'], 
        'loan_count': [4308, 10814, 4014, 2335], 
       'debt': [403, 782, 370, 186]}
purpose_df = pd.DataFrame(data_purpose_df)
purpose_df['non_return'] = purpose_df['debt'] / purpose_df['loan_count']
display(purpose_df)

,family_id,loan_count,debt,non_return
0,операции с автомобилем,4308,403,0.093547
1,операции с недвижимостью,10814,782,0.072314
2,получение образования,4014,370,0.092177
3,проведение свадьбы,2335,186,0.079657


##### Вывод 4:

В этой категории заемщиков самыми надежными являются те кто имеет цель операции с недвижимостью. В ютой категории всего около 7.2% процентов невозвратов, в то время как кредиты выданные на операции с автомобилем и получение образования являются самыми проблемными и имеют просрочку порядка 9.3% и 9.2% соответственно.

In [ ]:
report_family_status = data.pivot_table(
    index='family_status',
    values='debt',
    aggfunc=['sum', 'count']
)

In [ ]:
report_family_status

,sum,count
,debt,debt
family_status,,
Не женат / не замужем,274,2813
в разводе,85,1195
вдовец / вдова,63,960
гражданский брак,388,4177
женат / замужем,931,12380


In [ ]:
# удаляем лишний. уровень агрегации debt
report_family_status.columns = report_family_status.columns.droplevel(1)

In [ ]:
report_family_status

,sum,count
family_status,,
Не женат / не замужем,274,2813
в разводе,85,1195
вдовец / вдова,63,960
гражданский брак,388,4177
женат / замужем,931,12380


In [ ]:
# добавляем расчет вероятности не возврата кредита
report_family_status['prob_debt'] = report_family_status['sum'] / report_family_status['count']

In [ ]:
report_family_status

,sum,count,prob_debt
family_status,,,
Не женат / не замужем,274,2813,0.097405
в разводе,85,1195,0.071130
вдовец / вдова,63,960,0.065625
гражданский брак,388,4177,0.092890
женат / замужем,931,12380,0.075202


## Общий вывод:

Очевидно, наиболее благоприятными для банка являются целевые кредиты на ОПЕРАЦИИ С НЕДВИЖИМОСТЬЮ, в которых заемщик имеет доход от 30000 да 50000. Более странным является благоприятный для банка семейный статус: в разводе либо вдова/вдовец, так как эти катгорий являются наиболее надежными. 

При оценке категории заемщиков в соответствии с количеством детей, наиболее информативными будут категории либо без детей либо с 1-2 ребенком, так как количество кредитов среди других категорий ничтожно мало и их оценка врядли даст достоверную информацию. Исходя из этого наиболее благоприятным для банка является заемщик без детей, так как в этой категории показатель невозвратов на примерно 2% меньше чем у заемщиков с 1-2 ребенком. 

В тоже время банку следует обратить повышенное внимание на кредиты связанные с автомобилями и получением образования, особенно если заемщиком выступает не женатый(не замужний человек) с доходом либо менее 300000 либо от 50000 до 200000, так как вероятность возврата такого кредита значительно ниже.